# **Scraping Amazon**

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
link = "https://www.amazon.in/s?k=dishwasher&crid=1K49P42QD14P0&sprefix=%2Caps%2C924&ref=nb_sb_ss_recent_10_0_recent"

headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0",
    "Accept-Language": "en-US,en;q=0.9",
}

webpage = requests.get(link, headers=headers)
webpage.text

In [ ]:
yum = BeautifulSoup(webpage.text, 'lxml')

data = []

for link_tag in yum.find_all('a', attrs={'class': 'a-link-normal s-no-outline'})[8:-4]:
    full_url = 'https://amazon.in' + link_tag.get('href').strip()
    response = requests.get(full_url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch {full_url}")
        continue

    soup = BeautifulSoup(response.text, 'lxml')

    title_tag = soup.find('span', attrs={'id': 'productTitle'})
    title = title_tag.text.strip() if title_tag else "N/A"



    price = "N/A"
    for price_tag in soup.find_all('span', attrs={'class': 'a-offscreen'}):
        price = price_tag.text.strip()
        break



    brand = "N/A"
    brand_cell = soup.find('td', class_='a-span9')
    if brand_cell:
        brand_span = brand_cell.find('span', class_='a-size-base po-break-word')
        if brand_span:
            brand = brand_span.text.strip()


    data.append({
        'Title': title,
        'Price': price,
        'Brand': brand
    })

    print(f"Scraped: {title}")


df = pd.DataFrame(data)
df.to_csv('Scraped-amazon.csv', index=False)